### Model Training Pipeline

This notebook will be very similar to notebook **10_lightgbm_with_hyperparameter_tuning**. The major difference is that we will be reading the data from the feature store instead of a local parquet file. We will need the config.py variables so we will start by loading that in.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import config file for HOPSWORKS vars
import src.config as config

Connect to the project and get our feature store. Since we have already created the feature store, we do not need to include the description, primary keys, and event time keys as we previously did.

In [3]:
import hopsworks

# connect to our project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# get feature store
feature_store = project.get_feature_store()

# get feature group
feature_group = feature_store.get_or_create_feature_group(
    name = config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1049751
Connected. Call `.close()` to terminate connection gracefully.


##### Feature View

Next, we create the feature view. We code this so that we can create the feature view if it is not set up yet and if it is set up, we just get it.

A feature view is a set of features that come from one or more feature groups. In the feature view we can join features from multiple feature groups into a final dataset. A feature view is metadata which basically means "data about data" that organizes and manages the data we give it.

In [5]:
# create feature view if it is not already set up
try:
    feature_store.create_feature_view(
        name = config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already created.')

# get feature view whether it was created before, or now
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=1
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1049751/fs/1041478/fv/time_series_hourly_feature_view/version/1


##### Create Training Data

From our feature view we will call the **training_data** method. This method will give us back two variables. The first variable is the actual time series training data. The second variable is more metadata about the data that include things such as names, datatypes, and statistics. Since we wont be using them we use **_** which tells python to ignore this information so that we only receive the training data.

In [6]:
ts_data, _ = feature_view.training_data(
    description='Hourly time series data'
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (19.22s) 
2024-09-26 10:15:46,129 WARNING: VersionWarning: Incremented version to `1`.



In [7]:
ts_data

,pickup_hour,rides,pickup_location_id
0,2024-03-26 11:00:00+00:00,1,212
1,2023-04-02 11:00:00+00:00,0,101
2,2022-11-22 02:00:00+00:00,0,149
3,2023-03-06 15:00:00+00:00,0,185
4,2023-06-03 18:00:00+00:00,163,164
...,...,...,...
5183640,2024-02-22 11:00:00+00:00,0,133
5183641,2023-10-07 09:00:00+00:00,0,192
5183642,2023-03-16 06:00:00+00:00,0,78
5183643,2022-03-08 01:00:00+00:00,0,40


In [13]:
# sort this data 
ts_data.sort_values(by = ['pickup_location_id', 'pickup_hour'], inplace = True)
ts_data

,pickup_hour,rides,pickup_location_id
851055,2022-01-01 00:00:00+00:00,0,1
2818255,2022-01-01 01:00:00+00:00,0,1
2010330,2022-01-01 02:00:00+00:00,0,1
672294,2022-01-01 03:00:00+00:00,0,1
4911277,2022-01-01 04:00:00+00:00,1,1
...,...,...,...
4692207,2024-09-26 11:00:00+00:00,5,265
4692529,2024-09-26 12:00:00+00:00,3,265
4692733,2024-09-26 13:00:00+00:00,4,265
4692953,2024-09-26 14:00:00+00:00,2,265


##### Features and Targets

The next step is to create the features and targets for our training data as we did previously.

In [17]:
# get features and targets
from src.data import transform_ts_data_into_features_and_target

features, target = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28,  # one month of hourly data
    step_size=23  # so not every hour is first hour
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

  0%|          | 0/265 [00:00<?, ?it/s]

2024-09-26 10:53:31,828 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



  0%|          | 1/265 [00:00<00:53,  4.90it/s]

2024-09-26 10:53:32,039 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



  1%|          | 2/265 [00:00<01:10,  3.73it/s]

...

2024-09-26 10:54:08,388 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



 60%|█████▉    | 158/265 [00:36<00:29,  3.58it/s]

2024-09-26 10:54:08,654 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



 60%|█████▉    | 158/265 [00:37<00:25,  4.27it/s]


KeyboardInterrupt: 